In [1]:
import pandas as pd
import numpy as np

import dask
import dask.dataframe as dd

In [2]:
from datetime import datetime

In [3]:
formatted_dataset = dd.read_csv('formatted_dataset_3.csv')

In [4]:
top_20_records = formatted_dataset.head(20)
top_20_records

,IP,request_date,request_time,request_hour,request_minute,request_second,request_day_of_week,type_of_server,requested_bytes
0,54.36.149.41,2019-01-22,03:56:14,3,56,14,Tuesday,filter,30577
1,31.56.96.51,2019-01-22,03:56:16,3,56,16,Tuesday,image,5667
2,31.56.96.51,2019-01-22,03:56:16,3,56,16,Tuesday,image,5379
3,40.77.167.129,2019-01-22,03:56:17,3,56,17,Tuesday,image,1696
4,91.99.72.15,2019-01-22,03:56:17,3,56,17,Tuesday,product,41483
5,40.77.167.129,2019-01-22,03:56:17,3,56,17,Tuesday,image,2654
6,40.77.167.129,2019-01-22,03:56:18,3,56,18,Tuesday,image,3688
7,40.77.167.129,2019-01-22,03:56:18,3,56,18,Tuesday,image,14776
8,66.249.66.194,2019-01-22,03:56:18,3,56,18,Tuesday,filter,34277
9,40.77.167.129,2019-01-22,03:56:18,3,56,18,Tuesday,image,1695


In [5]:
# Removing this since this has imcomplete data
formatted_dataset_wr = formatted_dataset.loc[~((formatted_dataset['request_date'] == '2019-01-22') & (formatted_dataset['request_hour'] == 3))]

In [6]:
columns_to_drop = ['IP', 'request_day_of_week', 'request_hour', 'request_minute', 'request_second', 'request_day_of_week']
formatted_dataset_wr = formatted_dataset_wr.drop(columns=columns_to_drop)

In [7]:
formatted_dataset.dtypes

IP                     object
request_date           object
request_time           object
request_hour            int64
request_minute          int64
request_second          int64
request_day_of_week    object
type_of_server         object
requested_bytes         int64
dtype: object

In [8]:
formatted_dataset_wr['request_date_time'] = formatted_dataset_wr['request_date'].astype(str) + ' ' + formatted_dataset_wr['request_time'].astype(str)
formatted_dataset_wr[['request_date', 'request_time', 'request_date_time']].compute()

columns_to_drop = ['request_date', 'request_time']
formatted_dataset_wr = formatted_dataset_wr.drop(columns=columns_to_drop)

In [9]:
formatted_dataset_wr['request_date_time_object'] = dd.to_datetime(formatted_dataset_wr['request_date_time'], format='%Y-%m-%d %H:%M:%S')

In [10]:
# Preliminary Data Analysis

In [11]:
formatted_dataset_wr.dtypes


type_of_server                      object
requested_bytes                      int64
request_date_time                   object
request_date_time_object    datetime64[ns]
dtype: object

In [12]:
formatted_dataset_wr = formatted_dataset_wr.drop(columns=['request_date_time'])

In [13]:
# Segregating into different server types
# Image dataset
image_requests = formatted_dataset_wr.loc[formatted_dataset_wr['type_of_server'] == 'image']
image_requests = image_requests.drop(columns=['type_of_server'])
image_requests.to_csv('subsets/image_requests.csv', index=False, single_file=True)

# Static
static_requests = formatted_dataset_wr.loc[formatted_dataset_wr['type_of_server'] == 'static']
static_requests = static_requests.drop(columns=['type_of_server'])
static_requests.to_csv('subsets/static_requests.csv', index=False, single_file=True)


['/Users/fyrozdadapeer/Desktop/IIITKota Mtech Course work/3. Coursework/6. Open Ended Project/Project Files/subsets/static_requests.csv']

In [14]:
# Filter & Product
filter_product_requests_servers = ['filter', 'product']

# filter_product_requests = formatted_dataset.loc[formatted_dataset['type_of_server'] == 'filter' or formatted_dataset['type_of_server'] == 'product']
filter_product_requests = formatted_dataset_wr.loc[formatted_dataset_wr['type_of_server'].isin(filter_product_requests_servers)]
filter_product_requests = filter_product_requests.drop(columns=['type_of_server'])
filter_product_requests.to_csv('subsets/filter_product_requests.csv', index=False, single_file=True)

['/Users/fyrozdadapeer/Desktop/IIITKota Mtech Course work/3. Coursework/6. Open Ended Project/Project Files/subsets/filter_product_requests.csv']

In [15]:
# Settings
settings_requests = formatted_dataset_wr.loc[formatted_dataset_wr['type_of_server'] == 'settings']
settings_requests = settings_requests.drop(columns=['type_of_server'])
settings_requests.to_csv('subsets/settings_requests.csv', index=False, single_file=True)

['/Users/fyrozdadapeer/Desktop/IIITKota Mtech Course work/3. Coursework/6. Open Ended Project/Project Files/subsets/settings_requests.csv']

In [16]:
# Other main functionalities
other_requests_servers_excluded = ['image', 'static', 'filter', 'product', 'settings']
other_requests = formatted_dataset_wr.loc[~formatted_dataset_wr['type_of_server'].isin(other_requests_servers_excluded)]
other_requests = other_requests.drop(columns=['type_of_server'])
other_requests.to_csv('subsets/other_requests.csv', index=False, single_file=True)

['/Users/fyrozdadapeer/Desktop/IIITKota Mtech Course work/3. Coursework/6. Open Ended Project/Project Files/subsets/other_requests.csv']